In [1]:
import pyroomacoustics as pra

import os
from tqdm import tqdm

import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.autograd import profiler
import torchaudio

from einops import rearrange

from src.dataset import SignalDataset, TRUNetDataset
from src.loss import loss_tot, loss_MR, loss_MR_w
from models.fspen import FullSubPathExtension 

from IPython.display import Audio

from scipy.io.wavfile import write

import matplotlib.pyplot as plt

In [2]:
# DATA_DIR = os.path.join("..", "data", "musan", "speech")
# TRAIN_DIR = os.path.join(DATA_DIR, "train_chunks")
# TEST_DIR = os.path.join(DATA_DIR, "test_chunks")

DATA_DIR = os.path.join("data", "real")

CHKP_DIR = os.path.join("checkpoints")
np.set_printoptions(precision=3)
torch.set_printoptions(precision=3)

In [3]:
N_FFTS = 512
HOP_LENGTH = 256
SR = 48_000

DEVICE = "cpu" # torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"It's {DEVICE} time!!!")

It's cpu time!!!


In [4]:
dataset = TRUNetDataset(DATA_DIR, sr=48_000, return_noise=False, return_rir=False)

In [5]:
from src.fspen_configs import TrainConfig, TrainConfigLarge, TrainConfigRNN, TrainConfigRNN1, TrainConfigRNN2

configs = TrainConfigLarge()
print(sum(configs.bands_num_in_groups), configs.dual_path_extension["num_modules"])
fspen = FullSubPathExtension(configs=configs)# .to(DEVICE)

state_d = torch.load(os.path.join(CHKP_DIR, "fspen_chkp", "fspen_voicebank_subband_loss_causal#2.pt"), weights_only=False)

# model = FullSubPathExtension(nfft=512, hop=256)
# fspen.load_state_dict(state_d["model_state_dict"])

32 2


In [6]:
fspen.load_state_dict(state_d["model_state_dict"])

<All keys matched successfully>

In [7]:
def vorbis_window(winlen, device="cuda"):
    sq = torch.sin(torch.pi/2*(torch.sin(torch.pi/winlen*(torch.arange(winlen)-0.5))**2)).float()
    return sq

In [8]:
file_paths = os.listdir(DATA_DIR)

In [9]:
PCS = torch.ones(257, device=DEVICE)      # Perceptual Contrast Stretching
PCS[0:3] = 1
PCS[3:6] = 1.070175439
PCS[6:9] = 1.182456140
PCS[9:12] = 1.287719298
PCS[12:138] = 1.4       # Pre Set
PCS[138:166] = 1.322807018
PCS[166:200] = 1.238596491
PCS[200:241] = 1.161403509
PCS[241:256] = 1.077192982

In [10]:
window = vorbis_window(512)

for x in tqdm(file_paths):
    # input_sig, gt, gt_noise, gt_rir = dataset[i]
    signal, signal_sr = torchaudio.load(os.path.join(DATA_DIR, x))
    
    # signal = signal / torch.max(torch.abs(signal))
    
    spec = torch.stft(
            signal,
            n_fft=N_FFTS,
            hop_length=HOP_LENGTH,
            # onesided=True,
            win_length=512,
            window=window,
            return_complex=True,
            normalized=True,
            center=True
        ) 
    
    input_spec_pcs = PCS[:, None, None] * torch.transpose(torch.log1p(torch.abs(spec)), 1, 0)
    input_spec_pcs = torch.transpose(input_spec_pcs, 1, 0)
    input_spec_pcs = torch.polar(input_spec_pcs, spec.angle())
    abs_spectrum_pcs = input_spec_pcs.abs()
    # print(torch.transpose(torch.log1p(torch.abs(input_spec)), 1, 0).dtype)
    input_spec_pcs = torch.permute(torch.view_as_real(input_spec_pcs), dims=(0, 2, 3, 1))
    batch, frames, channels, frequency = input_spec_pcs.shape

    abs_spectrum_pcs = torch.permute(abs_spectrum_pcs, dims=(0, 2, 1))
    abs_spectrum_pcs = torch.reshape(abs_spectrum_pcs, shape=(batch, frames, 1, frequency))
    
    output, _ = fspen(input_spec_pcs, abs_spectrum_pcs)
    
    output = torch.permute(output, dims=(0, 3, 1, 2))
    output[..., 0] = torch.expm1(output[..., 0])
    output = torch.view_as_complex(output)
    
    window = vorbis_window(N_FFTS)# .to(device)
    # print(output_d.shape, input_spec.shape)
    out_wave = torch.istft(output, n_fft=N_FFTS, hop_length=HOP_LENGTH, win_length=N_FFTS,
                           window=window,
                           # onesided=True,
                           return_complex=False,
                           normalized=True,
                           center=True)
    out_wave = out_wave.reshape(-1)
    write(f'notebooks/real_test/{x}', SR, out_wave.cpu().detach().numpy())
    

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [01:06<00:00, 11.08s/it]
